In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [3]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [14]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.pool.register_forward_hook(layer1_hook)

In [15]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

D:\ProgramData\Anaconda3\envs\py39\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [16]:
outputs

[tensor([[[[8.0113e-01, 5.1395e-01, 8.2509e-01,  ..., 5.3390e-01,
            5.1693e-01, 1.4663e+00],
           [1.2692e+00, 1.0161e+00, 5.3385e-01,  ..., 1.6258e+00,
            5.5302e-01, 9.7337e-01],
           [1.3282e+00, 6.3114e-01, 1.9452e+00,  ..., 7.6703e-01,
            7.5836e-01, 4.7593e-01],
           ...,
           [8.9028e-01, 5.1972e-01, 4.4591e-01,  ..., 5.2974e-01,
            8.8370e-01, 5.3777e-01],
           [5.4918e-01, 9.5559e-01, 1.5423e+00,  ..., 0.0000e+00,
            3.9883e-01, 7.5150e-01],
           [4.6713e-01, 2.6153e-01, 9.4395e-01,  ..., 5.6067e-01,
            2.0434e-01, 6.2545e-01]],
 
          [[7.5330e-01, 1.4035e-01, 2.8079e-01,  ..., 9.2379e-01,
            1.4306e+00, 4.2717e-01],
           [1.2802e+00, 3.9792e-01, 6.8885e-01,  ..., 1.2829e+00,
            6.7196e-01, 4.7270e-01],
           [1.0568e+00, 5.9532e-01, 5.7908e-01,  ..., 0.0000e+00,
            5.6091e-01, 5.5862e-01],
           ...,
           [4.2189e-01, 3.6593e-01, 5.

In [17]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [18]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            nn.init.ones_(m.bias)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [19]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


